<a href="https://colab.research.google.com/github/timsetsfire/wandb-examples/blob/main/colab/WandB_with_Scala_via_py4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WandB with Scala via py4j

This notebook is meant to run in google colab.  

This is just a proof of concept and is not supported by W&B.  

grab and install some necessary stuff 👇

In [ ]:
%%capture
!git clone https://github.com/timsetsfire/urban-barnacle.git
!cd /content/urban-barnacle/src/main/python && make &> /dev/null
!pip install /content/urban-barnacle/src/main/python/dist/wandb_gateway-*-py3-none-any.whl -q
!curl https://pjreddie.com/media/files/mnist_train.csv > mnist_train.csv
!curl https://pjreddie.com/media/files/mnist_test.csv > mnist_test.csv

## Install the Scala Kernel
If you get a "scala" kernel not recognized warning when loading up the notebook for the first time, start by running the two cells below. Once you are done **reload the page** to load the notebook in the installed Scala kernel.

In [ ]:
%%shell
SCALA_VERSION=2.12.8 ALMOND_VERSION=0.3.0+16-548dc10f-SNAPSHOT
curl -Lo coursier https://git.io/coursier-cli
chmod +x coursier
./coursier bootstrap \
    -r jitpack -r sonatype:snapshots \
    -i user -I user:sh.almond:scala-kernel-api_$SCALA_VERSION:$ALMOND_VERSION \
    sh.almond:scala-kernel_$SCALA_VERSION:$ALMOND_VERSION \
    --sources --default=true \
    -o almond-snapshot --embed-files=false &> /dev/null
rm coursier
./almond-snapshot --install --global --force &> /dev/null
rm almond-snapshot

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42577  100 42577    0     0  47465      0 --:--:-- --:--:-- --:--:--  659k


In [ ]:
%%shell
echo "{
  \"language\" : \"scala\",
  \"display_name\" : \"Scala\",
  \"argv\" : [
    \"bash\",
    \"-c\",
    \"env LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libpython3.6m.so:\$LD_PRELOAD java -jar /usr/local/share/jupyter/kernels/scala/launcher.jar --connection-file {connection_file}\"
  ]
}" > /usr/local/share/jupyter/kernels/scala/kernel.json

# Refresh Browser

## Download Necessary Libraries

Using the import statements below will download all libraries necessary to create the MNist Model.  

FYI - The below cell generates tons of stdout.

In [ ]:
import $ivy.`com.google.code.gson:gson:2.9.0`

In [ ]:
import $ivy.`org.nd4j:nd4j-native-platform:1.0.0-M2`
import $ivy.`org.deeplearning4j:deeplearning4j-datasets:1.0.0-M2`
import $ivy.`org.deeplearning4j:deeplearning4j-core:1.0.0-M2`
import $ivy.`org.nd4j:nd4s_2.11:1.0.0-beta7`
import $ivy.`org.nd4j:nd4j-api:1.0.0-M2`
import $ivy.`org.nd4j:nd4j-native-platform:1.0.0-M2`
// import $ivy.`org.datavec:datavec-api:1.0.0-M2`
// import $ivy.`org.datavec:datavec-data-image:1.0.0-M2`
// import $ivy.`org.datavec:datavec-local:1.0.0-M2`
// import $ivy.`org.deeplearning4j:deeplearning4j-datasets:1.0.0-M2`
// import $ivy.`org.nd4j:nd4j-cuda-7.5-platform:1.0.0-M2`

// https://deeplearning4j.konduit.ai/spark/tutorials/dl4j-on-spark-quickstart
// import $ivy.`org.deeplearning4j:dl4j-spark-parameterserver_2.11:1.0.0-M2`

In [2]:

  import org.deeplearning4j.nn.conf.MultiLayerConfiguration;
  import org.deeplearning4j.nn.conf.NeuralNetConfiguration;
  import org.deeplearning4j.nn.conf.layers.DenseLayer;
  import org.deeplearning4j.nn.conf.layers.OutputLayer;
  import org.deeplearning4j.nn.multilayer.MultiLayerNetwork;
  import org.deeplearning4j.nn.weights.WeightInit;
  import org.deeplearning4j.optimize.listeners.ScoreIterationListener;
  import org.nd4j.evaluation.classification.Evaluation;
  import org.nd4j.linalg.activations.Activation;
  import org.nd4j.linalg.dataset.api.iterator.DataSetIterator;
  import org.nd4j.linalg.learning.config.{Nadam, Adam};
  import org.nd4j.linalg.lossfunctions.LossFunctions.LossFunction;
  import org.nd4j.linalg.dataset.DataSet;
  import org.nd4j.linalg.factory.Nd4j
  import java.io.{File, PrintWriter}
  import org.datavec.api.records.reader.RecordReader;
  import org.datavec.api.records.reader.impl.csv.CSVRecordReader;
  import org.datavec.api.split.FileSplit;
  import org.deeplearning4j.datasets.datavec.RecordReaderDataSetIterator;


import org.deeplearning4j.nn.conf.MultiLayerConfiguration;
  
import org.deeplearning4j.nn.conf.NeuralNetConfiguration;
  
import org.deeplearning4j.nn.conf.layers.DenseLayer;
  
import org.deeplearning4j.nn.conf.layers.OutputLayer;
  
import org.deeplearning4j.nn.multilayer.MultiLayerNetwork;
  
import org.deeplearning4j.nn.weights.WeightInit;
  
import org.deeplearning4j.optimize.listeners.ScoreIterationListener;
  
import org.nd4j.evaluation.classification.Evaluation;
  
import org.nd4j.linalg.activations.Activation;
  
import org.nd4j.linalg.dataset.api.iterator.DataSetIterator;
  
import org.nd4j.linalg.learning.config.{Nadam, Adam};
  
import org.nd4j.linalg.lossfunctions.LossFunctions.LossFunction;
  
import org.nd4j.linalg.dataset.DataSet;
  
import org.nd4j.linalg.factory.Nd4j
  
import java.io.{File, PrintWriter}
  
import org.datavec.api.records.reader.RecordReader;
  
import org.datavec.api.records.reader.impl.csv.CSVRecordReader;
  
import org.datavec.api.split.FileSplit;


## Functions for loading data

* `loadData` will load the data into dataset iterators
* `loadData2` loads data directly to a dataset (we'll use this one)

In [3]:
def loadData(path: String, batchSize: Int, labelIndex: Int, numClasses: Int): DataSetIterator = {
    val file = new java.io.File(path)
    val reader: RecordReader = new CSVRecordReader()
    reader.initialize(new FileSplit( file ))
    new RecordReaderDataSetIterator(reader, batchSize, labelIndex, numClasses); //reader, batch size, label index, num of label classes
}

def loadData2(path: String, labelIndex: Int, numClasses: Int) = {
  val data = Nd4j.readNumpy(path, ",")
  val labels = data.getColumn(labelIndex)
  val Array(m,n) = data.shape
  val features = data.getColumns(  (0 until n.toInt).filter( _ != labelIndex):_*)
  val oneHotLabels = Nd4j.zeros(m,numClasses) 
  labels.dup.data.asInt.zipWithIndex.foreach { 
    case (k,v) => oneHotLabels.putScalar(v,k,1.0)
  }
  val dataset = new DataSet(features, oneHotLabels)
  dataset
}

defined function loadData
defined function loadData2

In [4]:
// add the wandb scala bindings
{{
  val myJar = "/content/urban-barnacle/lib/py4j-wandb-assembly-0.1.0.jar"
  val path = java.nio.file.FileSystems.getDefault().getPath(myJar)
  val x = ammonite.ops.Path(path)
  interp.load.cp(x)
}}

## Weights & Biases

implementation wasn't thought out to carefully.  Everything is glued together via py4j and a java interface and some ugly scala laid on top

very limited bindings available

* `login`
* `init` -  when called, creates a run on the python side.
* `log` - logs detail with the run
* `logArtifact` - currently only works with files and directories
* `finish` - finishes the run.  

TODO
* `useArtifact` 
* `logMedia`

Example 

```
import py4j.wandb._ 
import py4j.wandb.Implicits._
val wandb = new WandB("wandb-session.log")
wandb.login(key=API_KEY)
wandb.init(project = "myproject")
val data = Map[String, Any]("metric" -> 0.1)
wandb.log(data) 
wandb.finish
```


In [5]:
import py4j.wandb._ 
import py4j.wandb.Implicits._
// currently using default ports, need to assign dynamically
val wandb = new WandB("wandb-session.log") // stdout will be logged to wandb-session.log

import py4j.wandb._ 

import py4j.wandb.Implicits._
// currently using default ports, need to assign dynamically

wandb: WandB = py4j.wandb.WandB@5f09f13e

In [6]:
val projectName = "dl4j-scala-colab-nov"

projectName: String = "dl4j-scala-colab-nov"

In [7]:
// if this get calls too soon after creating the WandB instance
// it will error -> need to give time for python server to startup
wandb.login(key="")

res6: java.lang.Boolean = true

In [8]:
val test = loadData2("mnist_test.csv", 0, 10)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


test: DataSet = ===========INPUT===================
[[         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
  ..., 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0]]
=================OUTPUT==================
[[         0,         0,         0,  ...    1.0000,         0,         0], 
 [         0,         0,    1.0000,  ...         0,         0,         0], 
 [         0,    1.0000,         0,  ...         0,         0,         0], 
  ..., 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,    

In [ ]:
val testFeatures = test.getFeatures

testFeatures: org.nd4j.linalg.api.ndarray.INDArray = [[         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
  ..., 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0]]

In [10]:
import java.util.HashMap
import java.util.LinkedList
import scala.collection.JavaConverters._

import java.util.HashMap

import java.util.LinkedList

import scala.collection.JavaConverters._

## Log Training and Test Data

In [12]:
wandb.init(
  project = projectName, 
  job_type = "data", 
  notes = "register train and test data")

val train = loadData2("mnist_train.csv", 0, 10)
val test = loadData2("mnist_test.csv", 0, 10)

wandb.logArtifact(
  "https://pjreddie.com/media/files/mnist_train.csv", 
  name="train-data", 
  artifactType = "dataset")
wandb.logArtifact(
  "https://pjreddie.com/media/files/mnist_test.csv",
  name="test-data", 
  artifactType = "dataset")

wandb.finish()

res11_0: java.lang.Boolean = true
train: DataSet = ===========INPUT===================
[[         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
  ..., 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0]]
=================OUTPUT==================
[[         0,         0,         0,  ...         0,         0,         0], 
 [    1.0000,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
  ..., 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,       

## Setup Config for Model Training

## Create the WandB Listener

To report metrics back to WandB

In [13]:
import org.deeplearning4j.optimize.api.BaseTrainingListener;
import java.io.Serializable;
import org.deeplearning4j.nn.api.Model;
import org.nd4j.linalg.dataset.DataSet;

case class WandBListener(logIteration: Int = 10,
                         testDataset: DataSet,
                         run: WandB) extends BaseTrainingListener with Serializable {

  if(logIteration <= 0) throw new Exception(s"Iteration must be greater than 0")

  override def iterationDone(model: Model, iteration: Int, epoch: Int): Unit = { 
    if(iteration % logIteration == 0) {
      val trainingScore = model.score();
      val testScore = model.asInstanceOf[MultiLayerNetwork].score(testDataset)
      val data = Map[String, Any](
        "epoch" -> epoch, "iteration" -> iteration, 
        "test_metric" -> testScore, "train_metric" -> trainingScore
        )
      run.log(data = data, step = iteration)
      print(s"Score on train dataset at iteration $iteration is $trainingScore")
      println(s"\t|| Score on test dataset at iteration $iteration is $testScore")
    }
  }
}

import org.deeplearning4j.optimize.api.BaseTrainingListener;

import java.io.Serializable;

import org.deeplearning4j.nn.api.Model;

import org.nd4j.linalg.dataset.DataSet;


defined class WandBListener

## Set up model configuration and Tags

In [14]:
import scala.collection.JavaConverters._
val rate=0.0035
val numEpochs=100
val numColumns=28
val randomSeed=123
val numRows=28
val outputNum=10

val wandbConfig = Map[String, Any](
  "numRows" -> numRows, "numColumns" -> numColumns, "outputNum" -> outputNum,
  "randomSeed" -> randomSeed, "numEpochs" -> numEpochs, "rate" -> rate
  )
val wandbTags = List("scala", "dl4j").asJava

import scala.collection.JavaConverters._

rate: Double = 0.0035
numEpochs: Int = 100
numColumns: Int = 28
randomSeed: Int = 123
numRows: Int = 28
outputNum: Int = 10
wandbConfig: Map[String, Any] = Map(
  "rate" -> 0.0035,
  "numEpochs" -> 100,
  "numColumns" -> 28,
  "randomSeed" -> 123,
  "numRows" -> 28,
  "outputNum" -> 10
)
wandbTags: java.util.List[String] = SeqWrapper(List("scala", "dl4j"))

In [15]:
val modelConf = new NeuralNetConfiguration.Builder()
            .seed(randomSeed) //include a random seed for reproducibility
            .activation(Activation.RELU)
            .weightInit(WeightInit.XAVIER)
            .updater(new Adam(rate))
            .l2(rate * 0.0001) // regularize learning model
            .list()
            .layer(new DenseLayer.Builder() //create the first input layer.
                    .nIn(numRows * numColumns)
                    .nOut(40)
                    .build())
            .layer(new DenseLayer.Builder() //create the second input layer
                    .nIn(40).activation(Activation.IDENTITY)
                    .nOut(10)
                    .build())
            .layer(new OutputLayer.Builder(LossFunction.MCXENT) //create hidden layer
                    .activation(Activation.SOFTMAX)
                    .nOut(outputNum)
                    .build())
            .build();

modelConf: MultiLayerConfiguration = {
  "backpropType" : "Standard",
  "cacheMode" : "NONE",
  "confs" : [ {
    "cacheMode" : "NONE",
    "dataType" : "FLOAT",
    "epochCount" : 0,
    "iterationCount" : 0,
    "layer" : {
      "@class" : "org.deeplearning4j.nn.conf.layers.DenseLayer",
      "activationFn" : {
        "@class" : "org.nd4j.linalg.activations.impl.ActivationReLU",
        "max" : null,
        "negativeSlope" : null,
        "threshold" : null
      },
      "biasInit" : 0.0,
      "biasUpdater" : null,
      "constraints" : null,
      "gainInit" : 1.0,
      "gradientNormalization" : "None",
      "gradientNormalizationThreshold" : 1.0,
      "hasBias" : true,
      "hasLayerNorm" : false,
      "idropout" : null,
      "iupdater" : {
        "@class" : "org.nd4j.linalg.learning.config.Adam",
        "beta1" : 0.9,
        "beta2" : 0.999,
        "epsilon" : 1.0E-8,
        "learningRate" : 0.0035
      },
      "layerName" : "layer0",
      "nin" : 784,
      "no

## Create an instance of the model

In [16]:
val model = new MultiLayerNetwork(modelConf);
model.setListeners(new WandBListener(10, test, wandb));  //print and log score every 10 iterations
model.init();


model: MultiLayerNetwork = org.deeplearning4j.nn.multilayer.MultiLayerNetwork@62136929

## Start WandB Run for training

In [17]:
wandb.init(project=projectName, config=wandbConfig, tags=wandbTags)

res16: java.lang.Boolean = true

## Train the network

In [18]:
(1 to wandbConfig.get("numEpochs").get.asInstanceOf[Int] ).foreach{  i => model.fit(train) }

Score on train dataset at iteration 0 is 20.63893311146226	|| Score on test dataset at iteration 0 is 19.521246649197682
Score on train dataset at iteration 10 is 6.0593237358630425	|| Score on test dataset at iteration 10 is 4.96630897103765
Score on train dataset at iteration 20 is 2.077976340914818	|| Score on test dataset at iteration 20 is 1.8557681804897157
Score on train dataset at iteration 30 is 1.5914523972603138	|| Score on test dataset at iteration 30 is 1.574763250179781
Score on train dataset at iteration 40 is 1.4703714770220755	|| Score on test dataset at iteration 40 is 1.4798907498759668
Score on train dataset at iteration 50 is 1.3508075547657206	|| Score on test dataset at iteration 50 is 1.3689089226768623
Score on train dataset at iteration 60 is 1.2566948080773603	|| Score on test dataset at iteration 60 is 1.262206007881391
Score on train dataset at iteration 70 is 1.1734109758482336	|| Score on test dataset at iteration 70 is 1.1804021246742527
Score on train d

## Log Artifact - Model

In [19]:
model.save(new File("model.bin"));
wandb.logArtifact("model.bin", name="serialized-model", artifactType = "model")

res18_1: java.lang.Boolean = true

## Log Artifact - Model Config

In [20]:
val yamlFile = new File(s"model-conf-${model.hashCode}.yaml")
val modelYamlPW = new PrintWriter(yamlFile)
modelYamlPW.write(modelConf.toYaml)
modelYamlPW.close
wandb.logArtifact(yamlFile.getCanonicalPath, name="model-config", artifactType = "config")

yamlFile: File = model-conf-1645439273.yaml
modelYamlPW: PrintWriter = java.io.PrintWriter@22c65b96
res19_4: java.lang.Boolean = true

## Log Artifact - Confusion Matrix and Stuff

In [21]:
val eval = new Evaluation(10);
val output = model.output(test.getFeatures);
eval.eval(test.getLabels, output);

eval: Evaluation = 

========================Evaluation Metrics========================
 # of classes:    10
 Accuracy:        0.7080
 Precision:       0.7997
 Recall:          0.7037
 F1 Score:        0.7087
Precision, recall & F1: macro-averaged (equally weighted avg. of 10 classes)


=========================Confusion Matrix=========================
   0   1   2   3   4   5   6   7   8   9
-----------------------------------------
 881   0   4   1   1   1   5   1  84   2 | 0 = 0
   0 988   1   7   4   0   4   1 126   4 | 1 = 1
  13   0 719   6   0   8   7   5 247  27 | 2 = 2
   3   0  11 625   0 211   2   3 146   9 | 3 = 3
   0   0   2   1 228   1   4   2 109 635 | 4 = 4
   5   2   6 146   0 434   5   1 284   9 | 5 = 5
   8   2   2   2   3   6 727   0 187  21 | 6 = 6
   0   2  22   2  10   3   0 714 117 158 | 7 = 7
   3   1   8   6   0  22   1   1 900  32 | 8 = 8
   4   3   0   3   6  14   1   9 105 864 | 9 = 9

Confusion matrix format: Actual (rowClass) predicted as (columnClass) N

In [22]:
val cf = new File("results.txt")
val pw = new PrintWriter(cf)
pw.write(eval.toString)
pw.close 
wandb.logArtifact(cf.getCanonicalPath, name="metrics", artifactType="txt")

cf: File = results.txt
pw: PrintWriter = java.io.PrintWriter@3f4d0f87
res21_4: java.lang.Boolean = true

## Finish the run

In [23]:
wandb.finish

res22: java.lang.Boolean = true

## Shutdown Python Server

In [24]:
wandb.shutdownGateway

If you have made is this far, open `wandb-session.log` and review the output